<a href="https://colab.research.google.com/github/cchen744/olist-regional-customer-experience-analysis/blob/main/01_data_overview.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Load datasets

In [1]:
import os
import pandas as pd

os.environ['KAGGLE_USERNAME'] = "chenmengmengchen" # Replace with username from json
os.environ['KAGGLE_KEY'] = "KGAT_f146268b4a26741708c060f77c63a2b3"           # Replace with key from json

from kaggle.api.kaggle_api_extended import KaggleApi

# 1. Authenticate
api = KaggleApi()
api.authenticate()

# 2. Download and Extract
# This handles the "extraction" part automatically with unzip=True
dataset_slug = "olistbr/brazilian-ecommerce"
download_path = "./brazilian_ecommerce_data"

print(f"Downloading {dataset_slug}...")
api.dataset_download_files(dataset_slug, path=download_path, unzip=True)
print("Download and extraction complete.\n")

# 3. Load the data using os.listdir
# The Olist dataset has multiple CSV files (orders, customers, products, etc.)
dataframes = {}

print("Loading files...")
for filename in os.listdir(download_path):
    if filename.endswith(".csv"):
        # Construct full file path
        file_path = os.path.join(download_path, filename)

        # Create a simpler name for the dataframe (e.g., "olist_orders_dataset.csv" -> "orders")
        # This is optional but makes accessing them easier
        df_name = filename.split('_dataset')[0].replace('olist_', '')

        # Load into dictionary
        dataframes[df_name] = pd.read_csv(file_path)
        print(f"--> Loaded {filename} as '{df_name}' (Shape: {dataframes[df_name].shape})")

# Example: Accessing one of the loaded tables
print("\nExample: First 5 rows of orders:")

Dataset URL: https://www.kaggle.com/datasets/olistbr/brazilian-ecommerce
Download and extraction complete.

Loading files...
--> Loaded olist_order_payments_dataset.csv as 'order_payments' (Shape: (103886, 5))
--> Loaded olist_orders_dataset.csv as 'orders' (Shape: (99441, 8))
--> Loaded olist_order_reviews_dataset.csv as 'order_reviews' (Shape: (99224, 7))
--> Loaded olist_order_items_dataset.csv as 'order_items' (Shape: (112650, 7))
--> Loaded olist_products_dataset.csv as 'products' (Shape: (32951, 9))
--> Loaded olist_customers_dataset.csv as 'customers' (Shape: (99441, 5))
--> Loaded olist_geolocation_dataset.csv as 'geolocation' (Shape: (1000163, 5))
--> Loaded product_category_name_translation.csv as 'product_category_name_translation.csv' (Shape: (71, 2))
--> Loaded olist_sellers_dataset.csv as 'sellers' (Shape: (3095, 4))

Example: First 5 rows of orders:


In [2]:
for df_name, df in dataframes.items():
    print('\n',df_name,'\n', df.columns,'\n',df.shape)


 order_payments 
 Index(['order_id', 'payment_sequential', 'payment_type',
       'payment_installments', 'payment_value'],
      dtype='object') 
 (103886, 5)

 orders 
 Index(['order_id', 'customer_id', 'order_status', 'order_purchase_timestamp',
       'order_approved_at', 'order_delivered_carrier_date',
       'order_delivered_customer_date', 'order_estimated_delivery_date'],
      dtype='object') 
 (99441, 8)

 order_reviews 
 Index(['review_id', 'order_id', 'review_score', 'review_comment_title',
       'review_comment_message', 'review_creation_date',
       'review_answer_timestamp'],
      dtype='object') 
 (99224, 7)

 order_items 
 Index(['order_id', 'order_item_id', 'product_id', 'seller_id',
       'shipping_limit_date', 'price', 'freight_value'],
      dtype='object') 
 (112650, 7)

 products 
 Index(['product_id', 'product_category_name', 'product_name_lenght',
       'product_description_lenght', 'product_photos_qty', 'product_weight_g',
       'product_length_cm', 'pr

Join dataframes to link reviews to their geolocation.

In [3]:
order_reviews_df = dataframes['order_reviews']
orders_df = dataframes['orders']
customers_df = dataframes['customers']
geolocation_df = dataframes['geolocation'][['geolocation_zip_code_prefix', 'geolocation_city', 'geolocation_state']]

In [4]:
# join reviews to order
print(order_reviews_df.shape[0])
print(order_reviews_df['order_id'].nunique())
review_with_orders = pd.merge(order_reviews_df, orders_df, on='order_id')
print(review_with_orders.shape[0])


99224
98673
99224


In [5]:
# join orders with customers
print(orders_df.shape[0])
print(customers_df['customer_id'].nunique())
orders_with_customers = pd.merge(review_with_orders, customers_df, on='customer_id')

99441
99441


In [6]:
# final step: join geo location
print(orders_with_customers.shape[0])
print(orders_with_customers['customer_zip_code_prefix'].nunique())
final_df = pd.merge(orders_with_customers, geolocation_df, left_on='customer_zip_code_prefix', right_on='geolocation_zip_code_prefix')

99224
14973


In [7]:
review_with_orders["review_score"].max()

5

In [8]:
final_df.head()

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp,customer_id,order_status,order_purchase_timestamp,...,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,geolocation_zip_code_prefix,geolocation_city,geolocation_state
0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,NaN,NaN,2018-01-18 00:00:00,2018-01-18 21:46:59,41dcb106f807e993532d446263290104,delivered,2018-01-11 15:30:49,...,2018-01-12 21:57:22,2018-01-17 18:42:41,2018-02-02 00:00:00,68a5590b9926689be4e10f4ae2db21a8,6030,osasco,SP,6030,osasco,SP
1,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,NaN,NaN,2018-01-18 00:00:00,2018-01-18 21:46:59,41dcb106f807e993532d446263290104,delivered,2018-01-11 15:30:49,...,2018-01-12 21:57:22,2018-01-17 18:42:41,2018-02-02 00:00:00,68a5590b9926689be4e10f4ae2db21a8,6030,osasco,SP,6030,osasco,SP
2,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,NaN,NaN,2018-01-18 00:00:00,2018-01-18 21:46:59,41dcb106f807e993532d446263290104,delivered,2018-01-11 15:30:49,...,2018-01-12 21:57:22,2018-01-17 18:42:41,2018-02-02 00:00:00,68a5590b9926689be4e10f4ae2db21a8,6030,osasco,SP,6030,osasco,SP
3,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,NaN,NaN,2018-01-18 00:00:00,2018-01-18 21:46:59,41dcb106f807e993532d446263290104,delivered,2018-01-11 15:30:49,...,2018-01-12 21:57:22,2018-01-17 18:42:41,2018-02-02 00:00:00,68a5590b9926689be4e10f4ae2db21a8,6030,osasco,SP,6030,osasco,SP
4,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,NaN,NaN,2018-01-18 00:00:00,2018-01-18 21:46:59,41dcb106f807e993532d446263290104,delivered,2018-01-11 15:30:49,...,2018-01-12 21:57:22,2018-01-17 18:42:41,2018-02-02 00:00:00,68a5590b9926689be4e10f4ae2db21a8,6030,osasco,SP,6030,osasco,SP
